In [1]:
import pickle
import requests

In [40]:
# from https://cloud.gate.ac.uk/shopfront/displayItem/tagger-pos-pl-maxent1 - "A POS tagger for pl / Polish using the Universal 
# Dependencies POS tagset.
# This tagger is based on a simple maximum entropy model trained on the corpus from the universal dependencies collection using 
# the GATE Learning Framework plugin.
# The model is trained on all available corpora, except the test corpus. Evaluation on the test set gives 0.9456 accuracy. 
# Accuracy on out-of-vocabulary words (words not seen in the trainin set) is 0.9122 (case-sensitive) / 0.9174 (not 
# case-sensitive)."

URL = 'https://cloud-api.gate.ac.uk/process/tagger-pos-pl-maxent1'
PARAMS = {
    'Content-Type': 'text/plain',
    'Accept': 'application/json'
}
file = 'memoirs.txt'

In [33]:
def retrieve_text(filename, split_lines=True):
    with open(filename) as f:
        lines = f.readlines()
        
    if split_lines==True:
        return lines
    else:
        full_text = []
        for line in lines:
            line = line.strip('\n')
            full_text.append(line)
        full_text = " ".join(full_text)
        return full_text

In [41]:
text = retrieve_text(file)[0]
print(text)

Dziad mój Melchior urodzony roku 1741 we wsi Godziszewo parafii Rypnin syn komornika ziemskiego Jana Komornika ziemi Dobrrzyńskiej (a nie Antoniego – Antoni był starszym bratem ś.p. Melchiora – i nie rozumiem skąd się wzięła ta myłka, która później wdokumentach się powtarza) i matki Ewy z Pinińskich właścicieli Dóbr Strużewo z adlinencjami Puszczanki, Dąbrowy części, Ronantowizna, Żółtowizna, Będowszczyazna, Kmińszczyzna, Bęklowizna, Ruszkowizna. A że śp Dziad mój był najmłodszy i piąty z pomiędzy Rodzeństwa – odstąpił swoją część najstarszemu bratu swojemu Mateuszowi komornikowi ziemi Dobrzyńskiej za ośm tysięcy tynfów. Byli jeszcze bracia Wawrzyniec, Szymon oraz siostra Panna Urszula Czermińska. Stryjów miał śp. Melchior czterech i jedną ciotkę – Antoniego Wawrzyńca ożenionego z Głuską z niej trzech synów Jakuba, Łukasza i Floyrana. Maciej z Chądzyńską, Floryan z Korytyńską, Wawrzyniec z Muczynowską – mieli syna Ludwika – Ciotka za Rzotoławskim – wszystkich braci stryjecznych śp. Mel

In [45]:
# needs testing
class TaggerAnnotations:
    def __init__(self, url, headers, data):
        r = requests.post(url=url, data=data.encode('utf-8'), headers=headers) 
        self.annotations = r.json()
        
        tagger_annotations = []
        for entry in results['entities']['Token']:
            word = entry['string']
            pos = entry['upos']
            confidence = entry['LF_confidence']
            target_list = entry['LF_target_list']
            confidence_list = entry['LF_confidence_list']

            annotation = [word, pos, confidence, target_list, confidence_list]
            tagger_annotations.append(annotation)
        self.processed_annotations = tagger_annotations
        
    def print_annotations(self):
        for entry in self.processed_annotations:
            print(entry[0] + ' | ' + entry[1] + ' | ' + str(entry[2]))
        

In [46]:
tagger_anns = TaggerAnnotations(URL, PARAMS, text)

In [47]:
tagger_anns.print_annotations()

Dziad | NOUN | 0.7880786605621269
mój | DET | 0.2910872818352366
Melchior | PROPN | 0.9882690037441527
urodzony | ADJ | 0.9838059834469768
roku | NOUN | 0.9893036802508473
1741 | ADJ | 0.8396864437394121
we | ADP | 0.8037721315042852
wsi | NOUN | 0.9555202276615747
Godziszewo | PROPN | 0.8356249362998734
parafii | NOUN | 0.8281642944816713
Rypnin | PROPN | 0.9761848889223718
syn | ADP | 0.44610724347153724
komornika | NOUN | 0.9921092663869862
ziemskiego | ADJ | 0.8841559648575619
Jana | PROPN | 0.9862540896222082
Komornika | PROPN | 0.9672533951057602
ziemi | VERB | 0.49472188150793944
Dobrrzyńskiej | PROPN | 0.6015370726961496
( | PUNCT | 0.9918210601889236
a | CCONJ | 0.9402611173855633
nie | PART | 0.9244890288853184
Antoniego | PROPN | 0.9069279540341488
– | PUNCT | 0.9999364511646827
Antoni | PROPN | 0.991452151960567
był | AUX | 0.9803619371237606
starszym | ADJ | 0.9942088789940817
bratem | NOUN | 0.9645196816687935
ś | X | 0.41795764186248896
. | PUNCT | 0.9999997391795165
p |

In [48]:
tagger_anns.annotations

{'text': 'Dziad mój Melchior urodzony roku 1741 we wsi Godziszewo parafii Rypnin syn komornika ziemskiego Jana Komornika ziemi Dobrrzyńskiej (a nie Antoniego – Antoni był starszym bratem ś.p. Melchiora – i nie rozumiem skąd się wzięła ta myłka, która później wdokumentach się powtarza) i matki Ewy z Pinińskich właścicieli Dóbr Strużewo z adlinencjami Puszczanki, Dąbrowy części, Ronantowizna, Żółtowizna, Będowszczyazna, Kmińszczyzna, Bęklowizna, Ruszkowizna. A że śp Dziad mój był najmłodszy i piąty z pomiędzy Rodzeństwa – odstąpił swoją część najstarszemu bratu swojemu Mateuszowi komornikowi ziemi Dobrzyńskiej za ośm tysięcy tynfów. Byli jeszcze bracia Wawrzyniec, Szymon oraz siostra Panna Urszula Czermińska. Stryjów miał śp. Melchior czterech i jedną ciotkę – Antoniego Wawrzyńca ożenionego z Głuską z niej trzech synów Jakuba, Łukasza i Floyrana. Maciej z Chądzyńską, Floryan z Korytyńską, Wawrzyniec z Muczynowską – mieli syna Ludwika – Ciotka za Rzotoławskim – wszystkich braci stryjeczny

In [49]:
tagger_anns.processed_annotations

[['Dziad',
  'NOUN',
  0.7880786605621269,
  ['NOUN',
   'PROPN',
   'VERB',
   'PART',
   'ADJ',
   'ADV',
   'PRON',
   'ADP',
   'X',
   'AUX',
   'NUM',
   'CCONJ',
   'DET',
   'SCONJ',
   'PUNCT'],
  [0.7880786605621269,
   0.10489510292170405,
   0.027720336902426913,
   0.018035291525443625,
   0.017980053962304694,
   0.013517882831340243,
   0.01216509694709733,
   0.0060562273826352195,
   0.004075824690180677,
   0.0022669842614183424,
   0.001946130303838891,
   0.0013286172956452696,
   0.0010223203841544658,
   0.000901971103838763,
   9.49892584455281e-06]],
 ['mój',
  'DET',
  0.2910872818352366,
  ['DET',
   'VERB',
   'AUX',
   'CCONJ',
   'NOUN',
   'PART',
   'ADJ',
   'ADP',
   'ADV',
   'PRON',
   'SCONJ',
   'X',
   'NUM',
   'PROPN',
   'PUNCT'],
  [0.2910872818352366,
   0.27297554868377827,
   0.21909272484437967,
   0.053556895383268466,
   0.05203179616442586,
   0.04100637533155515,
   0.02861265548432886,
   0.015236516204534414,
   0.009614096521121718,


In [50]:
import nltk


In [51]:
nltk.tokenize.word_tokenize(text)

['Dziad',
 'mój',
 'Melchior',
 'urodzony',
 'roku',
 '1741',
 'we',
 'wsi',
 'Godziszewo',
 'parafii',
 'Rypnin',
 'syn',
 'komornika',
 'ziemskiego',
 'Jana',
 'Komornika',
 'ziemi',
 'Dobrrzyńskiej',
 '(',
 'a',
 'nie',
 'Antoniego',
 '–',
 'Antoni',
 'był',
 'starszym',
 'bratem',
 'ś.p',
 '.',
 'Melchiora',
 '–',
 'i',
 'nie',
 'rozumiem',
 'skąd',
 'się',
 'wzięła',
 'ta',
 'myłka',
 ',',
 'która',
 'później',
 'wdokumentach',
 'się',
 'powtarza',
 ')',
 'i',
 'matki',
 'Ewy',
 'z',
 'Pinińskich',
 'właścicieli',
 'Dóbr',
 'Strużewo',
 'z',
 'adlinencjami',
 'Puszczanki',
 ',',
 'Dąbrowy',
 'części',
 ',',
 'Ronantowizna',
 ',',
 'Żółtowizna',
 ',',
 'Będowszczyazna',
 ',',
 'Kmińszczyzna',
 ',',
 'Bęklowizna',
 ',',
 'Ruszkowizna',
 '.',
 'A',
 'że',
 'śp',
 'Dziad',
 'mój',
 'był',
 'najmłodszy',
 'i',
 'piąty',
 'z',
 'pomiędzy',
 'Rodzeństwa',
 '–',
 'odstąpił',
 'swoją',
 'część',
 'najstarszemu',
 'bratu',
 'swojemu',
 'Mateuszowi',
 'komornikowi',
 'ziemi',
 'Dobrzyńskiej'